In [ ]:
import pandas as pd
import numpy as np
%pip install instaloader
import instaloader

In [ ]:
USER = input("Введи логин от своего инстаграма")
PASSWORD = input("Введи пароль от своего инстаграма")
loader = instaloader.Instaloader()
loader.login(USER, PASSWORD)

In [ ]:
target_profile = input("Введи логин профиле в котором нужно сравнить подписчиков и подписки")
profile = instaloader.Profile.from_username(loader.context, target_profile)

In [ ]:
df_followers = pd.Series(profile.get_followers())
df_followees = pd.Series(profile.get_followees())

In [ ]:
inst_sub = pd.DataFrame({'followers': df_followers, 'followees': df_followees})

inst_sub['followers'] = inst_sub['followers'].astype(str)
inst_sub['followees'] = inst_sub['followees'].astype(str)
inst_sub = inst_sub.fillna({'followees': "0", 'followees': "0"})

def clean_data(inst_sub):
    # Split text using string ' ' in columns: 'followers', 'followees'
    loc_0 = inst_sub.columns.get_loc('followers')
    inst_sub_split = inst_sub['followers'].str.split(expand=True).add_prefix('followers_')
    inst_sub = pd.concat([inst_sub.iloc[:, :loc_0], inst_sub_split, inst_sub.iloc[:, loc_0:]], axis=1)
    loc_1 = inst_sub.columns.get_loc('followees')
    inst_sub_split = inst_sub['followees'].str.split(expand=True).add_prefix('followees_')
    inst_sub = pd.concat([inst_sub.iloc[:, :loc_1], inst_sub_split, inst_sub.iloc[:, loc_1:]], axis=1)
    inst_sub = inst_sub.drop(columns=['followers', 'followees'])
    # Drop columns: 'followers_0', 'followers_2' and 2 other columns
    inst_sub = inst_sub.drop(columns=['followers_0', 'followers_2', 'followees_0', 'followees_2'])
    # Rename column 'followers_1' to 'followers'
    inst_sub = inst_sub.rename(columns={'followers_1': 'followers'})
    # Rename column 'followees_1' to 'followees'
    inst_sub = inst_sub.rename(columns={'followees_1': 'followees'})
    return inst_sub

inst_sub_clean = clean_data(inst_sub.copy())

inst_sub_clean['followers'] = inst_sub_clean['followers'].astype(str)
inst_sub_clean['followees'] = inst_sub_clean['followees'].astype(str)
inst_sub_clean = inst_sub_clean.fillna({'followees': "0", 'followees': "0"})

inst_sub_clean.head()

In [ ]:
inst_sub_clean = pd.Series(np.setdiff1d(inst_sub_clean['followees'], inst_sub_clean['followers']))
inst_sub_clean

In [ ]:
inst_sub_clean.to_csv('clean_alsu.csv', index=False, header=False)